# Settings

In [1]:
from dotenv import load_dotenv
import os

# .env load 
load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "default"

# check API KEY 
if "OPENAI_API_KEY" not in os.environ:
    print("API KEY 정보가 없습니다. 확인 후 환경변수에 등록해주세요.")

# 1. Langchain 기초

## 1) Langchain 없이 

In [2]:
# Langchain 없이 Chatgpt 사용하기 - 지시문 이용
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "당신은 USER의 말에 공감해주는 상답사입니다."},
        {"role": "user", "content": "우울해서 빵을 샀어"}
    ]
)

response = completion.choices[0].message
response.content

'저도 때때로 우울할 때 먹는 것으로 위로를 받는 법을 알아요. 빵을 사서 조금이라도 기분이 나아졌으면 좋겠어요. 하지만 더 좋은 방법으로 우울함을 극복하는 방법도 찾아보는 것을 권장해요. 함께 하면 더 좋은 방법을 찾을 수 있을 거예요. 함께 화이팅해요!'

## 2) Langchain 사용

In [8]:
# Model의 구조
from langchain_openai import ChatOpenAI 
model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
print(model)

client=<openai.resources.chat.completions.Completions object at 0x00000147FF5671C0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000147FF5782E0> openai_api_key=SecretStr('**********') openai_proxy=''


### (1) 단순 질의응답

In [3]:
from langchain_openai import ChatOpenAI 

model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
response = model.invoke("오늘 너무 우울해서 빵을 샀어")
print(response.content)

빵을 산 것이 우울함을 달래줄 수 있기를 바랍니다. 그 빵을 맛있게 먹으면서 기분이 조금이라도 나아지길 기대해봐도 좋을 것 같아요. 그리고 우울한 마음을 털어놓고 이야기할 수 있는 친구나 가족과 소통하는 것도 도움이 될 수 있을 거예요. 힘내세요! 함께 응원해요.


### (2) 역할 구분 질의응답

In [9]:
from langchain_openai import ChatOpenAI 
from langchain.schema import AIMessage, HumanMessage, SystemMessage 

model = ChatOpenAI(model_name="gpt-3.5-turbo")
messages = [
    SystemMessage(
        content = "당신은 매우 이성적인 사람입니다."
    ),
    HumanMessage(
        content = "오늘 너무 우울해서 빵을 샀어"
    ),
]

response = model.invoke(messages)
response

AIMessage(content='우울할 때 당황스럽고 위압감을 느낄 수 있지만, 먹는 것으로 감정을 달래는 것은 일시적인 해결책이 될 수 있습니다. 좀 더 건강한 방법으로 우울함을 극복할 수 있는 방법을 찾아보는 것이 중요합니다. 운동을 하거나 취미 활동을 즐기는 것도 좋은 방법일 수 있습니다. 또한, 친구나 가족과 소통하거나 전문가의 도움을 받는 것도 고려해보세요. 심리적인 안정을 찾는 것이 중요합니다. 함께 이야기를 나누는 것이 도움이 되길 바랍니다.', response_metadata={'token_usage': {'completion_tokens': 222, 'prompt_tokens': 46, 'total_tokens': 268}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c152843a-fa60-4f5e-a3d2-b239777dbacc-0')

# 2. Langchain + Prompt

In [2]:
from langchain_openai import ChatOpenAI 
from langchain.chains import LLMChain

def make_chain(prompt):
    model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
    chain = LLMChain(prompt=prompt, llm=model)

    return chain

## 1) PromptTemplate

### EXAMPLE 1

In [4]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{object}가 영어로 뭐야?")
print(prompt)

input_variables=['object'] template='{object}가 영어로 뭐야?'


In [5]:
chain = make_chain(prompt)

response = chain.invoke({"object":"다람쥐"})
response

{'object': '다람쥐', 'text': 'squirrel'}

### EXAMPLE2

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{object}가 {language}로 뭐야?")
print(prompt)

input_variables=['language', 'object'] template='{object}가 {language}로 뭐야?'


In [9]:
chain = make_chain(prompt)

response = chain.invoke({"object":"다람쥐", "language":"영어"})
print(response)

{'object': '다람쥐', 'language': '영어', 'text': 'squirrel'}

## 2) ChatPromptTemplate

### EXAMPLE 1

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "영어로 번역해주세요"),
        ("human", "{object}"),
    ]
)
print(prompt)

input_variables=['object'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='영어로 번역해주세요')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['object'], template='{object}'))]


In [15]:
chain = make_chain(prompt)

response = chain.invoke({"object":"다람쥐"})
print(response)

{'object': '다람쥐', 'text': 'Squirrel'}


### EXAMPLE 2

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "{language}로 번역해주세요"),
        ("human", "{object}"),
    ]
)
print(prompt)

input_variables=['language', 'object'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='{language}로 번역해주세요')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['object'], template='{object}'))]


In [17]:
chain = make_chain(prompt)

response = chain.invoke({"object":"다람쥐", "language":"영어"})
response

{'object': '다람쥐', 'language': '영어', 'text': 'squirrel'}

## 3) 반복 작업

In [29]:
input_list = [{"object":"선풍기"},{"object":"지하철"},{"object":"번역"}]

In [33]:
prompt1 = PromptTemplate.from_template("{object} 가 영어로 뭐야?")
prompt2 = ChatPromptTemplate.from_messages(
    [
        ("system", "영어로 번역해주세요"),
        ("human", "{object}"),
    ]
)

### EXAMPLE 1 - apply

In [34]:
# Prompt 1
chain = make_chain(prompt1)

response = chain.apply(input_list)
print(response)


[{'text': '선풍기는 영어로 "fan" 입니다.'},
 {'text': 'Subway'},
 {'text': '번역은 "translation" 입니다.'}]

In [35]:
# Prompt 2
chain = make_chain(prompt2)

response = chain.apply(input_list)
print(response)

[{'text': 'fan'}, {'text': 'subway'}, {'text': 'Translation'}]

### EXAMPLE 2 - generate

In [45]:
# Prompt 1
chain = make_chain(prompt1)

response = chain.generate(input_list)
print([x[0].text for x in response.generations])

['선풍기는 영어로 "fan" 입니다.', 'Subway', '번역은 영어로 translation이라고 합니다.']


In [46]:
# Prompt 2
chain = make_chain(prompt2)

response = chain.generate(input_list)
print([x[0].text for x in response.generations])

['fan', 'Subway', 'Translation']


# 3. LCEL

In [60]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("{input}에 대해 한국어로 한 줄로 설명해줘")
model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
output_parser = StrOutputParser()
chain = prompt | model | output_parser
print(chain)

first=PromptTemplate(input_variables=['input'], template='{input}에 대해 한국어로 한 줄로 설명해줘') middle=[ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000147876BB580>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000147876D4F70>, openai_api_key=SecretStr('**********'), openai_proxy='')] last=StrOutputParser()


## 1) Method

### (1) invoke / ainvoke

In [62]:
import time
 
# 동기 실행 함수 - invoke
def run_sync():
    start_time = time.time()
    result1 = chain.invoke({"input": "Python"})
    result2 = chain.invoke({"input": "JavaScript"})
    result3 = chain.invoke({"input": "Rust"})
    end_time = time.time()
    print(f"Sync execution time: {end_time - start_time:.2f} seconds")
    print(f"Result 1: {result1}")
    print(f"Result 2: {result2}")
    print(f"Result 3: {result3}")

run_sync()

Sync execution time: 3.83 seconds
Result 1: Python은 간결하고 가독성이 좋은 프로그래밍 언어로, 다양한 분야에서 사용되는 인기 있는 언어이다.
Result 2: JavaScript는 웹페이지를 동적으로 만들어주는 프로그래밍 언어이다.
Result 3: Rust는 안전하고 병행성이 뛰어난 시스템 프로그래밍 언어이다.


In [66]:
import time 
import asyncio

# 비동기 실행 함수 - ainvoke
async def run_async():
    start_time = time.time()
    tasks = [
        chain.ainvoke({"input": "Python"}),
        chain.ainvoke({"input": "JavaScript"}),
        chain.ainvoke({"input": "Rust"})
    ]
    results = await asyncio.gather(*tasks)
    end_time = time.time()
    print(f"Async execution time: {end_time - start_time:.2f} seconds")
    for i, result in enumerate(results, 1):
        print(f"Result {i}: {result}")

await run_async()

Async execution time: 1.77 seconds
Result 1: 파이썬은 간결하고 읽기 쉬운 문법을 가진 인기 있는 프로그래밍 언어이다.
Result 2: JavaScript는 웹페이지를 동적으로 제어하고 상호작용을 가능하게 하는 프로그래밍 언어이다.
Result 3: Rust는 안전하고 고성능인 시스템 프로그래밍 언어로, 메모리 안전성을 보장하면서도 추상화 수준을 제공하여 개발 생산성을 높이는 것을 목표로 하는 프로그래밍 언어이다.


### (2) batch / abatch

In [77]:
input_list = [{"input": "Python"}, {"input": "JavaScript"}, {"input": "Rust"}, {"input": "Flutter"}, {"input": "Spring"}]

In [80]:
import time 

# 동기 실행 코드 - batch
start_time = time.time()
result = chain.batch(input_list)
end_time = time.time()
print(f"Async execution time: {end_time - start_time:.2f} seconds")

Async execution time: 1.67 seconds


In [81]:
import time 

# 비동기 실행 코드 - batch
start_time = time.time()
result = chain.abatch(input_list)
await result
end_time = time.time()
print(f"Async execution time: {end_time - start_time:.2f} seconds")

Async execution time: 1.54 seconds


In [56]:
chain.stream({"input":"파이썬"}) # generator로 출력되는 것을 확인

<generator object RunnableSequence.stream at 0x000001478549B370>

### (3) stream / astream

In [89]:
for chunk in chain.stream({"input":"파이썬"}):
    print(chunk, end="", flush=True)

파이썬은 간결하고 읽기 쉬운 문법을 가진 프로그래밍 언어로 다양한 분야에서 활용되는 인기 있는 언어이다.

In [90]:
async for chunk in chain.astream({"input":"파이썬"}):
    print(chunk, end="", flush=True)

파이썬은 간결하고 쉽게 배울 수 있는 프로그래밍 언어로, 다양한 분야에서 활용되는 인기 있는 언어이다.

In [106]:
stream = chain.astream_log({"input":"파이썬"})
async for chunk in stream:
    print(chunk)
    print("="*100)


RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'af9b9fd5-8252-4c8b-a6d0-7ebbec5b1413',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
RunLogPatch({'op': 'add',
  'path': '/logs/PromptTemplate',
  'value': {'end_time': None,
            'final_output': None,
            'id': '2a9e28e3-ba58-423b-8e85-47d1d026a94b',
            'metadata': {},
            'name': 'PromptTemplate',
            'start_time': '2024-06-26T04:16:34.208+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'prompt'}})
RunLogPatch({'op': 'add',
  'path': '/logs/PromptTemplate/final_output',
  'value': StringPromptValue(text='파이썬에 대해 한국어로 한 줄로 설명해줘')},
 {'op': 'add',
  'path': '/logs/PromptTemplate/end_time',
  'value': '2024-06-26T04:16:34.218+00:00'})
RunLogPatch({'op': 'add',
  'path': '/logs/Chat

### (4) 병렬 처리

In [107]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel 

model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
output_parser = StrOutputParser()

prompt1 = PromptTemplate.from_template("{country}의 주요 언어를 알려줘")
prompt2 = PromptTemplate.from_template("{country}의 대표적인 랜드마크 3개를 알려줘")

chain1 = prompt1 | model | output_parser
chain2 = prompt2 | model | output_parser

combined = RunnableParallel(
    language = chain1,
    landmarks = chain2
)
print(combined)

steps={'language': PromptTemplate(input_variables=['country'], template='{country}의 주요 언어를 알려줘')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014787882980>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014787752EF0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), 'landmarks': PromptTemplate(input_variables=['country'], template='{country}의 대표적인 랜드마크 3개를 알려줘')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014787882980>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014787752EF0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()}


In [108]:
combined.invoke({"country":"한국"})

{'language': '한국의 주요 언어는 한국어입니다. 한국어는 대부분의 한국 사람들이 사용하는 언어로, 국내에서는 공식 언어로 사용되고 있습니다. 또한, 영어도 많은 사람들이 학습하고 사용하고 있으며, 중국어와 일본어도 일부 지역에서 사용되고 있습니다.',
 'landmarks': '1. 남산타워 - 서울의 대표적인 랜드마크로서, 서울 시내와 한강을 한눈에 볼 수 있는 전망대가 유명하다.\n2. 경복궁 - 서울에 위치한 조선 시대의 궁궐로서, 아름다운 전통 한옥 건물과 근정전, 경회루 등을 볼 수 있다.\n3. 부산 타워 - 부산의 랜드마크로서, 부산 시내와 해안도로를 한눈에 볼 수 있는 전망대와 야경이 유명하다.'}

In [109]:
country_list = [{"country":"한국"},{"country":"미국"},{"country":"일본"}]
combined.batch(country_list)

[{'language': '한국의 주요 언어는 한국어입니다. 한국어는 대한민국과 북한에서 사용되는 공식 언어로, 한반도 전역에서 널리 사용되고 있습니다. 또한 영어도 상당히 흔히 사용되는 언어 중 하나입니다.',
  'landmarks': '1. 경복궁: 서울에 위치한 조선 시대 궁궐로, 한국의 대표적인 궁궐 중 하나로 유명하다.\n2. 롯데월드 타워: 서울 송파구에 위치한 123층으로 이루어진 대한민국 최고의 랜드마크 건물이다.\n3. 부산 해운대 해수욕장: 부산 해운대구에 위치한 대한민국에서 가장 유명한 해변으로, 국내외 관광객들이 많이 찾는 곳이다.'},
 {'language': '미국의 주요 언어는 영어입니다. 미국에서는 영어가 공식 언어로 사용되며, 대다수의 미국인들이 영어를 사용하고 있습니다. 또한, 스페인어, 중국어, 프랑스어, 독일어 등 다양한 언어들도 미국에서 사용되고 있습니다.',
  'landmarks': '1. 자유의 여신상 (Statue of Liberty) - 뉴욕 하버에 위치한 자유의 여신상은 미국의 상징적인 랜드마크로, 자유와 민주주의를 상징합니다.\n\n2. 화이트 하우스 (The White House) - 워싱턴 D.C.에 위치한 미국 대통령의 공식 거처인 화이트 하우스는 미국 정부의 중심지로서 국가의 상징이기도 합니다.\n\n3. 그랜드 캐년 (Grand Canyon) - 아리조나주에 있는 그랜드 캐년은 자연의 위대함을 보여주는 대자연의 랜드마크로, 세계적으로 유명한 관광지입니다.'},
 {'language': '일본의 주요 언어는 일본어입니다. 일본어는 일본에서 사용되는 공용어로, 일본의 주요 언어로서 일본 국민 대다수가 일본어를 사용합니다. 일본어는 히라가나, 가타카나, 한자로 구성되어 있으며, 일본어의 발음은 한국어와는 다른 독특한 발음을 가지고 있습니다. 일본어는 일본에서의 의사소통을 위해 필수적인 언어이기 때문에, 일본을 방문하거나 일본과 관련된 업무를 하는 사람들은 일본어를 배우는 것이 도움이 됩니다.',
  'la

## 2) Runnable

### (1) RunnablePassthrough

In [118]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate.from_template("{object}가 영어로 뭐야?")
model = ChatOpenAI(model_name = "gpt-3.5-turbo") 
output_parser = StrOutputParser()

runnable = {"object": RunnablePassthrough()}
chain = runnable | prompt | model | output_parser 
chain.invoke("다람쥐")

'squirrel'

In [123]:
add_runnable = RunnablePassthrough.assign(new_object = lambda x: x["object"] + "인형")
add_runnable.invoke({"object": "다람쥐"})
# chain = add_runnable | prompt | model | chain 
# chain.invoke({"object": "다람쥐"})

{'object': '다람쥐', 'new_object': '다람쥐인형'}

In [21]:
# Langchain 이용하기 - 지시문 + 제시어 (LCEL)
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
    {language}에 대해 FORMAT에 맞춰 설명해주세요. 요약은 1줄로 표현해주세요.

    FORMAT:
    - 요약
    - 장점
    - 단점
"""
prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

response = chain.invoke({"language":"python"})
print(response)

- Python은 다양한 영역에서 사용되는 간결하고 읽기 쉬운 프로그래밍 언어이다.
- 장점: 문법이 간결하고 다양한 라이브러리와 도구를 제공하여 개발 속도가 빠르고 생산성이 높다.
- 단점: 실행 속도가 다른 언어에 비해 상대적으로 느릴 수 있으며, GIL(Global Interpreter Lock)로 인해 멀티스레딩이 효율적으로 이루어지지 않을 수 있다.


In [60]:
# Langchain 이용하기 - 지시문 + 제시어 (LCEL) (Chain 이해하기)
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
    {language}에 대해 FORMAT에 맞춰 설명해주세요. 요약은 1줄로 표현해주세요.

    FORMAT:
    - 요약
    - 장점
    - 단점
"""
prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

# Step 1. keyword Dict 입력 {"language":"python"}

# Step 2. Prompt에 전달 -> prompt_value
prompt_value = prompt.format_prompt(language="python")
print(prompt_value)

# Step 3. Model에 전달 -> output
output = model.invoke(prompt_value)
print(output)

# Step 4. String으로 출력 -> response
response = output_parser.invoke(output)
print(response)

text='\n    python에 대해 FORMAT에 맞춰 설명해주세요. 요약은 1줄로 표현해주세요.\n\n    FORMAT:\n    - 요약\n    - 장점\n    - 단점\n'
content='- 요약: Python은 인간 친화적인 문법과 다양한 라이브러리를 제공하여 다양한 분야에서 널리 사용되는 프로그래밍 언어이다.\n- 장점: 문법이 간결하고 읽기 쉽고, 다양한 라이브러리와 프레임워크가 존재하여 개발이 용이하며, 크로스 플랫폼 지원이 우수하다.\n- 단점: 성능이 다른 언어들에 비해 상대적으로 느리고, GIL(Global Interpreter Lock)로 인해 멀티스레드 처리가 제약을 받을 수 있다.' response_metadata={'token_usage': {'completion_tokens': 195, 'prompt_tokens': 63, 'total_tokens': 258}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None} id='run-e92c6ccf-e78c-4ab6-9bf7-4d3df942452d-0'
- 요약: Python은 인간 친화적인 문법과 다양한 라이브러리를 제공하여 다양한 분야에서 널리 사용되는 프로그래밍 언어이다.
- 장점: 문법이 간결하고 읽기 쉽고, 다양한 라이브러리와 프레임워크가 존재하여 개발이 용이하며, 크로스 플랫폼 지원이 우수하다.
- 단점: 성능이 다른 언어들에 비해 상대적으로 느리고, GIL(Global Interpreter Lock)로 인해 멀티스레드 처리가 제약을 받을 수 있다.


# Streaming

In [22]:
# Langchain 이용하기 - 단순 질문 + 출력을 Streaming : 터미널이나 콘솔에서 확인용으로 주로 쓰임
from langchain_openai import ChatOpenAI 
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]) 

response = model.invoke("파이썬에 대해 알려줘")
response

파이썬은 간단하고 쉽게 배울 수 있는 프로그래밍 언어로, 1991년에 발표된 고급 프로그래밍 언어입니다. 파이썬은 다양한 운영 체제에서 사용할 수 있으며, 다양한 분야에서 활용되고 있습니다. 

파이썬의 특징은 다음과 같습니다:
1. 간결하고 가독성이 좋은 문법
2. 다양한 라이브러리와 모듈을 지원하여 다양한 기능을 제공
3. 객체지향, 함수형, 절차지향 등 다양한 프로그래밍 스타일을 지원
4. 크로스 플랫폼 지원으로 다양한 운영 체제에서 사용 가능
5. 동적 타이핑을 지원하여 변수의 자료형을 명시적으로 선언하지 않아도 됨

파이썬은 데이터 분석, 인공지능, 웹 개발, 시스템 프로그래밍 등 다양한 분야에서 사용되고 있으며, 학습자들에게 많은 도움이 되는 인기 있는 프로그래밍 언어입니다. 파이썬을 배우면 다양한 분야에서 활용할 수 있는 능력을 기를 수 있습니다.

AIMessage(content='파이썬은 간단하고 쉽게 배울 수 있는 프로그래밍 언어로, 1991년에 발표된 고급 프로그래밍 언어입니다. 파이썬은 다양한 운영 체제에서 사용할 수 있으며, 다양한 분야에서 활용되고 있습니다. \n\n파이썬의 특징은 다음과 같습니다:\n1. 간결하고 가독성이 좋은 문법\n2. 다양한 라이브러리와 모듈을 지원하여 다양한 기능을 제공\n3. 객체지향, 함수형, 절차지향 등 다양한 프로그래밍 스타일을 지원\n4. 크로스 플랫폼 지원으로 다양한 운영 체제에서 사용 가능\n5. 동적 타이핑을 지원하여 변수의 자료형을 명시적으로 선언하지 않아도 됨\n\n파이썬은 데이터 분석, 인공지능, 웹 개발, 시스템 프로그래밍 등 다양한 분야에서 사용되고 있으며, 학습자들에게 많은 도움이 되는 인기 있는 프로그래밍 언어입니다. 파이썬을 배우면 다양한 분야에서 활용할 수 있는 능력을 기를 수 있습니다.', response_metadata={'finish_reason': 'stop'}, id='run-fa33ce5b-2c0c-4f43-aaf3-f2c0108bb1c2-0')

In [35]:
# Langchain 이용하기 - Streaming
from typing import Any, List, Dict, Union
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.agents import AgentAction, AgentFinish
from langchain_openai import ChatOpenAI 
from langchain_core.callbacks import BaseCallbackHandler

class CustomHandler(BaseCallbackHandler):
    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> Any:
        """Run when LLM starts running."""
        print("Run when LLM starts running")

    # def on_chat_model_start(self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs: Any) -> Any:
    #     """Run when Chat Model starts running."""

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        """Run on new LLM token. Only available when streaming is enabled."""
        print(token, end="")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        print("\nRun when LLM ends running")

    # def on_llm_error(
    #     self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    # ) -> Any:
    #     """Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> Any:
        """Run when chain starts running."""
        print("Run when chain starts running")

    # def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> Any:
    #     """Run when chain ends running."""

    # def on_chain_error(
    #     self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    # ) -> Any:
    #     """Run when chain errors."""

    # def on_tool_start(
    #     self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    # ) -> Any:
    #     """Run when tool starts running."""

    # def on_tool_end(self, output: Any, **kwargs: Any) -> Any:
    #     """Run when tool ends running."""

    # def on_tool_error(
    #     self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    # ) -> Any:
    #     """Run when tool errors."""

    # def on_text(self, text: str, **kwargs: Any) -> Any:
    #     """Run on arbitrary text."""

    # def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
    #     """Run on agent action."""

    # def on_agent_finish(self, finish: AgentFinish, **kwargs: Any) -> Any:
    #     """Run on agent end."""

model = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    streaming=True,
    callbacks=[CustomHandler()]) 

response = model.invoke("파이썬에 대해 알려줘")
response

Run when LLM starts running
파이썬은 간결하고 쉽게 배울 수 있는 프로그래밍 언어로, 1991년에 발표된 고급 프로그래밍 언어입니다. 파이썬은 다양한 운영 체제에서 실행할 수 있고, 다양한 목적으로 사용됩니다. 데이터 분석, 인공지능, 웹 개발 등의 분야에서 널리 사용되며, 다양한 라이브러리와 프레임워크를 지원하고 있습니다. 또한, 파이썬은 코드의 가독성이 좋고, 문법이 간결하여 빠르게 프로그램을 작성할 수 있습니다. 파이썬은 커뮤니티가 활발하고 다양한 자료와 도구가 제공되어 있어, 학습자들에게 많은 도움이 됩니다.
Run when LLM ends running


AIMessage(content='파이썬은 간결하고 쉽게 배울 수 있는 프로그래밍 언어로, 1991년에 발표된 고급 프로그래밍 언어입니다. 파이썬은 다양한 운영 체제에서 실행할 수 있고, 다양한 목적으로 사용됩니다. 데이터 분석, 인공지능, 웹 개발 등의 분야에서 널리 사용되며, 다양한 라이브러리와 프레임워크를 지원하고 있습니다. 또한, 파이썬은 코드의 가독성이 좋고, 문법이 간결하여 빠르게 프로그램을 작성할 수 있습니다. 파이썬은 커뮤니티가 활발하고 다양한 자료와 도구가 제공되어 있어, 학습자들에게 많은 도움이 됩니다.', response_metadata={'finish_reason': 'stop'}, id='run-f193a4ee-4eac-4f5f-b6ef-ab9047da08c5-0')

# Example 1. 지역 홍보대사

In [72]:
# Langchain 이용하기 - 지시문 + 제시어 (LCEL)
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
    당신은 대한민국 {location} 홍보대사입니다. FORMAT에 맞춰 홍보해주세요.
    환영인사와 좋은점은 20자 내외입니다.

    FORMAT:
    환영인사
    - 슬로건: 
    - 특산물: 
    - 관광지: 
    좋은점
"""
prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

response = chain.invoke({"location":"제주도"})
print(response)

환영합니다!
- 슬로건: "제주, 자연이 주는 휴식과 감동"
- 특산물: 감귤, 제주 흑돼지, 제주 삼다수
- 관광지: 성산일출봉, 용두암, 한라산 등
자연 속 휴식과 감동이 가득한 제주에서 즐거운 시간을 보내세요.


# Example 2. 청첩장 글귀 만들기

In [77]:
# Langchain 이용하기 - 지시문 + 제시어 (LCEL)
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
    TAG에 맞춰 결혼식 청첩장 글귀를 300자 내외로 만들어주세요. 
    문장마다 줄바꿈이 있어야 합니다.

    TAG: {tags}
"""
prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

response = chain.invoke({"tags":["장기 연애", "봄", "밝음"]})
print(response)

사랑하는 친구들과 함께하는 특별한 날,
긴 시간을 함께한 장기 연애의 결실을 맺는 순간.

봄이 활짝 피어나는 이 날,
사랑과 행복이 가득한 새로운 시작을 앞두고 있습니다.

따스한 햇살이 비추는 이 자리에서,
우리의 사랑을 함께 축복해 주실 수 있으신 분들을 초대합니다.

밝은 미래를 향해 함께하는 우리,
이 순간을 영원히 기억하며 더욱 행복한 인연으로 나아갈 것을 약속합니다.


# Example 3. 스트레칭 가이드

In [14]:
# Langchain 이용하기 - 지시문 + 제시어 (LCEL)
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
    {part}의 스트레칭을 도와주는 챗봇입니다. 
    스트레칭 방법을 1개만 단계별로 FORMAT에 맞춰 답변해주세요.
    몇 초간 몇 번 반복해야 하는지 자세하고 쉽게 알려주세요.

    FORMAT:
    스트레칭 명: 스트레칭 설명
    STEP1.
    STEP2.
    ...
"""
prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

response = chain.invoke({"part":"목"})
print(response)

스트레칭 명: 목 회전 스트레칭
STEP1. 목을 천천히 시계 반대 방향으로 돌리며 최대한 뒤로 굽힙니다. (10초 유지)
STEP2. 다시 원래 자세로 돌아옵니다.
STEP3. 이번에는 시계 방향으로 목을 돌리며 최대한 뒤로 굽힙니다. (10초 유지)
STEP4. 다시 원래 자세로 돌아옵니다.
STEP5. 총 3세트 반복해주세요.


# Example 4. 다이어트 레시피

In [90]:
# Langchain 이용하기 - 지시문 + 제시어 (LCEL)
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
    INGREDIENTS로 만들 수 있는 다이어트 레시피를 FORMAT에 맞춰 알려주세요.
    준비물은 무게 단위까지 자세히 작성해야 합니다.

    INGREDIENTS: {ingredients}
    
    FORMAT:
    요리명:
    준비물: 
    칼로리:
    방법:
    1. 
    2. 
    ...
"""
prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

response = chain.invoke({"ingredients":["단호박","우유","양송이"]})
print(response)

요리명: 다이어트 단호박 수프

준비물:
- 단호박 (500g)
- 우유 (200ml)
- 양송이 (100g)

칼로리: 약 150kcal

방법:
1. 먼저 단호박을 껍질을 벗기고 씻은 뒤 작게 썰어줍니다.
2. 양송이는 깨끗이 씻은 뒤 두툼하게 슬라이스합니다.
3. 냄비에 단호박과 양송이를 넣고 우유를 부어줍니다.
4. 뚜껑을 덮고 중불에서 약 20분간 끓여줍니다.
5. 모든 재료가 부드러워지면 믹서기에 갈아 부드러운 수프로 만들어줍니다.
6. 그릇에 담아 기호에 따라 소금이나 후추를 조금씩 넣어 맛을 조절한 뒤 완성합니다.


# Few Shot Learning

In [3]:
# Langchain 이용하기 - 지시문 + 제시어 (LCEL)
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# One Shot

template = """
    REVIEW의 긍/부정을 분류해주세요.

    REVIEW: {review}
"""
prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

response = chain.invoke({"review":"무난하게 재밌음그렇지만 무난함, 꼭 봐야될 정도는 아닌.."})
print(response)

부정적인 리뷰입니다.


In [12]:
# FewShot Learning
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Few Shot
examples = [
    {"review":"몇년째 나와도 재밌다.", "answer": "ANSWER: 긍정"},
    {"review":"판다야 너가 귀여운거 다해", "answer": "ANSWER: 긍정"},
    {"review":"중간에 잤어요 개연성이 없고 지루합니다 쿵푸팬더 팬들만 보세요", "answer": "ANSWER: 부정"}
]

example_prompt = PromptTemplate(
    input_variables=["review","answer"],
    template = "REVIEW: {review}\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix="REVIEW: {review}",
    input_variables=["review"]
)

review = "무난하게 재밌음그렇지만 무난함, 꼭 봐야될 정도는 아닌.."
final_prompt = prompt.format(review=review)
model = ChatOpenAI(model_name="gpt-3.5-turbo")

print(final_prompt)
print(model.invoke(final_prompt).content)

REVIEW: 몇년째 나와도 재밌다.
ANSWER: 긍정

REVIEW: 판다야 너가 귀여운거 다해
ANSWER: 긍정

REVIEW: 중간에 잤어요 개연성이 없고 지루합니다 쿵푸팬더 팬들만 보세요
ANSWER: 부정

REVIEW: 무난하게 재밌음그렇지만 무난함, 꼭 봐야될 정도는 아닌..
ANSWER: 중립


In [17]:
# Example Selector를 이용한 동적 Few-Shot learning
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

examples = [
    {"input": "토니모리 치크톤 젤리 블러셔", "output": "메이크업"},
    {"input": "마녀공장 퓨어 클렌징 오일", "output": "클렌징"},
    {"input": "식물나라 퀵앤딥 클렌징 티슈", "output": "클렌징"},
    {"input": "라운드랩 포 맨 자작나무 토너", "output": "스킨케어"},
    {"input": "아이오페 스템3 2종세트", "output": "스킨케어"}
]

example_prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = "Input: {input}\nOutput: {output}"
)


In [26]:
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1   
)

similar_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix="다음 제품의 카테고리를 분류해줘",
    suffix="Input: {input}\nOutput: ",
    input_variables=["input"]
)
model = ChatOpenAI(model_name="gpt-3.5-turbo")

final_prompt = similar_prompt.format(input="에스트라 아토베리어365크림")
print(response)
print(model.invoke(final_prompt).content)

다음 제품의 카테고리를 분류해줘

Input: 토니모리 치크톤 젤리 블러셔
Output: 메이크업

Input: 에스트라 아토베리어365크림
Output: 
스킨케어


In [9]:
# pip.req error
# https://github.com/ssut/py-hanspell/issues/12
# https://github.com/kdrkdrkdr/py-hanspell/blob/master/setup.py

from hanspell import spell_checker 

url = "https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy?passportKey=9174b407ac9ae3de9f16e75ddb56cb306aebd813&_callback=jQuery112408487938080478374_1713917132780&q=%EB%A7%9E%EC%B6%A4%EB%B2%95+%EA%B2%80%EC%82%AC%EB%A5%BC+%EC%9B%90%ED%95%98%EB%8A%94+%EB%8B%A8%EC%96%B4%EB%82%98+%EB%AC%B8%EC%9E%A5%EC%9D%84+%EC%9E%85%EB%A0%A5%ED%95%B4+%EC%A3%BC%EC%84%B8%EC%9A%94.&where=nexearch&color_blindness=0&_=1713917132781"
result = spell_checker.check("안녕하세요. 이문장은 한글 로 작성되 었 습니다.", url)
result.as_dict()

{'result': True,
 'original': '안녕하세요. 이문장은 한글 로 작성되 었 습니다.',
 'checked': '안녕하세요. 이 문장은 한글로 작성되었습니다.',
 'errors': 3,
 'words': OrderedDict([('안녕하세요.', 0),
              ('이', 2),
              ('문장은', 2),
              ('한글로', 2),
              ('작성되었습니다.', 2)]),
 'time': 0.6763484477996826}